# METHOD 1: Markov Chain Approach

## OBJECTIVE: Build a probabilistic Markov Chain model for urban flood prediction

### TASK DESCRIPTION:
Implement a Hidden Markov Model (HMM) or Markov Chain approach that:
1. Discretizes water levels into states (bins)
2. Learns transition probabilities between states
3. Uses rainfall as observation/forcing signal
4. Predicts future water level states autoregressively

### REQUIREMENTS:

1. STATE DISCRETIZATION:
   - Bin water levels into K states (e.g., K=20-50 bins)
   - Use quantile-based binning for balanced state distribution
   - Create separate state spaces for 1D and 2D nodes
   - Handle Model 1 (300m scale) and Model 2 (40m scale) separately

2. TRANSITION MATRIX LEARNING:
   For each node:
   - Compute P(state_t+1 | state_t, rainfall_t)
   - Use all training events to estimate probabilities
   - Consider rainfall-conditional transitions:
     * Low rain: P_low(s_t+1 | s_t)
     * Medium rain: P_med(s_t+1 | s_t)
     * High rain: P_high(s_t+1 | s_t)

3. SPATIAL COUPLING:
   - For 2D nodes: Include neighbor states in transition probability
     P(s_i,t+1 | s_i,t, s_neighbors,t, rainfall_t)
   - For 1D nodes: Include upstream pipe states
   - Use 1D-2D connections for cross-layer influence

4. PREDICTION MECHANISM:
   - Given initial state and rainfall forecast
   - Autoregressively sample/predict most likely state sequence
   - Convert final state bins back to continuous water levels
   - Use state bin centers or Gaussian smoothing

5. ADVANCED FEATURES:
   - Higher-order Markov: P(s_t+1 | s_t, s_t-1, s_t-2)
   - Rainfall history features: cumulative rain, peak intensity
   - Time-varying transitions for different event phases (rise vs recession)

### IMPLEMENTATION STEPS:

Step 1: Data Preparation (30 min)
- Load all training events
- Discretize water levels into states using quantiles
- Create rainfall bins (low/med/high) based on intensity
- Extract temporal sequences: [state_t, state_t+1, rainfall_t]

Step 2: Transition Matrix Estimation (1 hour)
- For each node:
  * Count transitions: C(s_i → s_j | rainfall_bin)
  * Normalize to probabilities: P = C / sum(C)
  * Add Laplace smoothing for unseen transitions
- Store matrices: transition_probs[node_idx][rainfall_bin]

Step 3: Spatial Extension (1 hour)
- For each 2D node, identify neighbors from edge_index
- Compute joint transition: P(s_i,t+1 | s_i,t, avg(s_neighbors,t))
- For 1D nodes, use upstream connectivity

Step 4: Prediction (30 min)
- Initialize with ground truth state at t=0
- Loop for each timestep:
  * Look up current state and rainfall bin
  * Sample next state from P(s_t+1 | s_t, rainfall)
  * OR use argmax for deterministic prediction
- Convert state sequence back to water levels

Step 5: Evaluation (30 min)
- Compute Standardized RMSE using competition metric
- Analyze where Markov chain fails (likely: extreme events)
- Compare to persistence baseline

### EXPECTED PERFORMANCE:
- Baseline: 0.1128
- Markov Chain Target: 0.08-0.10 (10-30% improvement)
- Strengths: Captures statistical patterns, fast inference
- Weaknesses: Discretization loss, limited memory, no complex spatial reasoning

### CODE SKELETON:


In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict

class MarkovChainFloodPredictor:
    def __init__(self, n_states=30, n_rainfall_bins=5):
        self.n_states = n_states
        self.n_rainfall_bins = n_rainfall_bins
        self.transition_probs = {}  # [node_idx][rainfall_bin]
        self.state_edges = None  # Bin edges for discretization
        
    def fit(self, water_levels, rainfall, node_idx):
        """
        water_levels: [n_timesteps, n_nodes]
        rainfall: [n_timesteps, n_2d_nodes]
        """
        # Discretize water levels into states
        self.state_edges = np.quantile(water_levels, 
                                       np.linspace(0, 1, self.n_states+1))
        states = np.digitize(water_levels, self.state_edges) - 1
        
        # Discretize rainfall
        rainfall_bins = self._bin_rainfall(rainfall)
        
        # Count transitions
        transition_counts = defaultdict(lambda: np.zeros((self.n_states, self.n_states)))
        
        for t in range(len(states) - 1):
            state_t = states[t, node_idx]
            state_t1 = states[t+1, node_idx]
            rain_bin = rainfall_bins[t]
            
            transition_counts[rain_bin][state_t, state_t1] += 1
        
        # Normalize to probabilities (with Laplace smoothing)
        for rain_bin in range(self.n_rainfall_bins):
            counts = transition_counts[rain_bin] + 1e-6
            self.transition_probs[(node_idx, rain_bin)] = counts / counts.sum(axis=1, keepdims=True)
    
    def predict(self, initial_state, rainfall_sequence, node_idx):
        """Predict water level sequence"""
        predictions = []
        current_state = np.digitize(initial_state, self.state_edges) - 1
        
        for rainfall_t in rainfall_sequence:
            rain_bin = self._bin_rainfall(rainfall_t)
            trans_prob = self.transition_probs.get((node_idx, rain_bin))
            
            # Sample next state (or argmax for deterministic)
            next_state = np.argmax(trans_prob[current_state])
            
            # Convert state back to water level (bin center)
            water_level = (self.state_edges[next_state] + 
                          self.state_edges[next_state+1]) / 2
            
            predictions.append(water_level)
            current_state = next_state
        
        return np.array(predictions)

    def _bin_rainfall(self, rainfall):
        # Placeholder for rainfall binning logic
        # You'll need to implement quantile based binning or similar here
        return np.zeros(len(rainfall), dtype=int) # Dummy return